In [19]:
# Standard
import numpy as np
import pandas as pd
from numpy.random import randn
from pandas import Series, DataFrame

# Stats
from scipy import stats
import math

# Plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
#plt.style.use('ggplot')

# Command so that plots appear in the iPython Notebook
%matplotlib inline

# Time access and conversions
import time

import random

#### 1. Gnerate x = a sequence of points, y = sin(x)+a small random error;
#### 2. Draw a scatter plot of x and y

In [39]:
df = DataFrame(randn(10000),columns=['x'])
df['sin(x)']=np.sin(df['x'])
df.head()

,x,sin(x)
0,0.216225,0.214544
1,0.011910,0.011910
2,-0.353065,-0.345775
3,1.225561,0.940996
4,-0.537939,-0.512367


In [41]:
df2 = DataFrame(randn(10000),columns=['a'])
df2.head()

,a
0,1.646489
1,-1.042926
2,-0.678927
3,1.617049
4,0.527821


In [47]:
df.add(df,fill_value=0)

,x,sin(x)
0,0.432450,0.429088
1,0.023820,0.023820
2,-0.706130,-0.691551
3,2.451123,1.881992
4,-1.075879,-1.024735
5,-1.154029,-1.091048
6,-2.024107,-1.695844
7,-1.637566,-1.460630
8,-2.616465,-1.931455
9,3.088051,1.999283


In [49]:
df['y'] = df.sin(x)+ df.a 
df.head()

AttributeError: 'DataFrame' object has no attribute 'sin'

#### 2. Draw a scatter plot of x and y

#### 3. Run a linear regression model to predict y, using only one feature--x;
#### print out the CV score of your model and the mathematical formula of your model;

#### 4. Draw a plot showing your predicted y, the real y, and ground truth--sin(x);

#### 5. Try to run a linear model using two features--x and x**2;
#### print out CV score and mathematical formula;

#### 6. Try linear models with feature from x, to x, x**2, x**3,... x**15, and plot the training score and validation score showing what happens when feature set gets larger;
#### Accoding to the result you get, what's the best number of features here;

#### 7. download a .xlsx file with multiple sheets nad read them all into pandas dataframe